In [3]:
import numpy as np
import pylab as plt
import swyft
import torch
from scipy import stats
%load_ext autoreload
%autoreload 2

In [4]:
DEVICE = 'cuda:0'

## Torus model

In [16]:
def model(params, center = np.array([0.6, 0.8])):
    a, b, c = params['a'], params['b'], params['c']
    r = ((a-center[0])**2+(b-center[1])**2)**0.5  # Return radial distance from center
    x = np.array([a, r, c])
    return dict(x=x)

def noise(obs, params, noise = np.array([0.03, 0.005, 0.2])):
    x = obs['x']
    n = np.random.randn(*x.shape)*noise
    return dict(x = x + n)

par0 = dict(a=0.57, b=0.8, c=1.0)
obs0 = model(par0)  # Using Asimov data

In [17]:
prior = swyft.Prior({"a": ["uniform", 0., 1.], "b": ["uniform",  0., 1.], "c": ["uniform", 0., 1.]})

In [36]:
cache = swyft.MemoryCache.from_simulator(model, prior)
posterior = swyft.amortize(model, prior, cache, noise = noise, N = 300)

Simulate:  22%|██▏       | 64/296 [00:00<00:00, 634.00it/s]

Creating new cache.
Adding 296 new samples. Run simulator!


Simulate: 100%|██████████| 296/296 [00:00<00:00, 629.50it/s]


n_features = 3
Start training
LR iteration 0
Validation loss: 0.0
Validation loss: 0.0
Total epochs: 2
LR iteration 1
Validation loss: 0.0
Validation loss: 0.0
Total epochs: 2
LR iteration 2
Validation loss: 0.0
Validation loss: 0.0
Total epochs: 2


## Inference

In [37]:
s = swyft.SWYFT(model, prior, noise = noise, obs = obs0, device = DEVICE)

Creating new cache.


In [40]:
s.focus(Ninit = 300, max_rounds=2)

Simulate:   0%|          | 0/1939 [00:00<?, ?it/s]

N = 300
Adding 1939 new samples. Run simulator!


Simulate: 100%|██████████| 1939/1939 [00:02<00:00, 783.61it/s]


n_features = 3
Start training
LR iteration 0
Validation loss: 3.2842125627729626
Validation loss: 3.2587495379977756
Validation loss: 3.1152200963762073
Validation loss: 3.196405039893256
Total epochs: 4
LR iteration 1
Validation loss: 3.1401646402147083
Validation loss: 3.1272286309136286
Validation loss: 3.0902068085140653
Validation loss: 3.0100533167521157
Validation loss: 3.047556506262885
Total epochs: 5
LR iteration 2
Validation loss: 2.968872759077284
Validation loss: 3.0888656510247126
Total epochs: 2
Volume shrinkage: 0.553974994473567
N = 383
No new simulator runs required.
No simulations required.
n_features = 3
Start training
LR iteration 0
Validation loss: 3.4005036883884006
Validation loss: 3.3326423168182373
Validation loss: 3.279906908671061
Validation loss: 3.2678186098734536
Validation loss: 3.194376760058933
Validation loss: 3.1967846287621393
Total epochs: 6
LR iteration 1
Validation loss: 3.1911941369374595
Validation loss: 3.1488317913479276
Validation loss: 3.06

In [7]:
s.posterior1d(N = 20000)

N = 20000
Round: 8
Adding 6832 new samples. Run simulator!


Simulate: 100%|██████████| 6832/6832 [00:08<00:00, 776.98it/s]


n_features = 3
Start training
LR iteration 0
Validation loss: 5.830409311479138
Validation loss: 5.817557819427982
Validation loss: 5.748897160253217
Validation loss: 5.700253140541815
Validation loss: 5.7414001803244314
Total epochs: 5
LR iteration 1
Validation loss: 5.566216991793725
Validation loss: 5.692911624908447
Total epochs: 2
LR iteration 2
Validation loss: 5.61087074587422
Validation loss: 5.544128752523853
Validation loss: 5.5616587977255545
Total epochs: 3


In [1]:
post = s.posteriors(n_samples = 10000000)
swyft.corner(post, ["a", "b", "c"], color='r', figsize = (15,15), truth=par0)

NameError: name 's' is not defined